# Prompt Template 提示词模板

## prompt template  字符模板

In [4]:
# 字符模板
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("你是一个{area}专家")
prompt.format(area="AI模型")

'你是一个AI模型专家'

# 对话模板 ChatPromptTemplate

In [ ]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
   ("system", "You are a helpful assistant."),
   ("ai", "I am an AI model."),
   ("human", "{user_input}"),
   ("assistant", "ai_response"),
  #  ("xxxx", "JJJJJJ")
  # Unexpected message type: xxxx.
  # Use one of 'human', 'user', 'ai', 'assistant', or 'system'.
])
chat_template.format_messages(user_input="Hello my AI!")

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I am an AI model.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello my AI!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='ai_response', additional_kwargs={}, response_metadata={})]

In [21]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

sm = SystemMessage(content='You are a helpful assistant.', additional_kwargs={'name': 'system'})
h_m = HumanMessage(content='You are a helpful assistant.', additional_kwargs={'name': 'human'})
ai_m = AIMessage(content='I am an AI model.')
[sm, h_m, ai_m]


[SystemMessage(content='You are a helpful assistant.', additional_kwargs={'name': 'system'}, response_metadata={}),
 HumanMessage(content='You are a helpful assistant.', additional_kwargs={'name': 'human'}, response_metadata={}),
 AIMessage(content='I am an AI model.', additional_kwargs={}, response_metadata={})]

## ChatMassagePromptTemplate

In [30]:
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import AIMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate

pt = "愿{obj}荣光永存！"

sys_template = SystemMessagePromptTemplate.from_template(template=pt)
sys_template.format(obj="AAA")

hum_template = HumanMessagePromptTemplate.from_template(template=pt)
hum_template.format(obj="BBB")

ai_template = AIMessagePromptTemplate.from_template(template=pt)
ai_template.format(obj="CCCC")

fat_template = ChatMessagePromptTemplate.from_template(role='father', template=pt)
fat_template.format(obj="DDDD")

# [sys_template, hum_template, ai_template, fat_template]

ChatMessage(content='愿DDDD荣光永存！', additional_kwargs={}, response_metadata={}, role='father')

## 自定义模板 StringPromptTemplate

In [3]:
# AI函数解析器
from langchain.prompts import StringPromptTemplate

# 定义一个简单的函数
def add(x: int, y: int) -> int:
    return x + y

# 创建一个自定义提示词模板
PROMPT="""\
你是一个函数解析器, 可以解析函数的参数和返回值。
根据给定的函数内容:{function_body},解析出一下内容：
参数列表, 返回值类型, 函数功能解释, 示例代码。
按照以下格式输出:

函数解析如下：
- 1. 函数名：{function_name}
- 2. 参数列表：
- 3. 返回值类型：
- 4. 函数体：
     {function_body}
- 5. 功能解释：
- 6. 示例代码：
"""

import inspect

def get_function_source(func):
    # 获取函数信息
    return inspect.getsource(func)


# 创建一个自定义的提示词模板类
class CustomPromptTemplate(StringPromptTemplate):
    def format(self, **kwargs):
        # 获得源代码信息
        func = kwargs.pop("function")
        func_info = get_function_source(func)
        return PROMPT.format(
            function_name=func.__name__,
            function_body=func_info,
        )


# 创建一个StringPromptTemplate实例
custom_pt = CustomPromptTemplate(input_variables=["function"])
cpt_str = custom_pt.format(function=add)

# 将生成的模板输入给AI
from langchain_ollama import ChatOllama

# 指定本地模型名称或路径
ollama_llm = ChatOllama(
    model="deepseek-r1:1.5b",  # 例如DeepSeek-R1 1.5B模型
    base_url="http://localhost:11434",  # Ollama默认地址
    temperature=0,
)
print(ollama_llm.predict(cpt_str))


<think>
嗯，我现在要解析这个函数。首先，函数名是add，参数都是int类型，返回值也是int。看起来这是一个简单的加法函数。

参数列表部分，我需要列出x和y的类型和位置。x在第一个位置，y在第二个位置，都是int型。

返回值类型很简单，就是int，直接写出来就可以了。

函数体的话，按照def的格式来写，参数用逗号分隔，后面加->，然后是return语句，把x+y写进去。

功能解释部分，我需要简要说明这个函数的作用，比如可以用来计算两个整数的和。这样用户就能明白它的作用了。

示例代码的话，我可以举个例子，比如add(3,4)应该返回7，或者add(-1,2)返回1。这样大家都能看到它怎么用。

现在把这些信息整理成格式化的输出吧。
</think>

函数解析如下：

- 1. 函数名：add
- 2. 参数列表：
  - x: int
  - y: int
- 3. 返回值类型：int
- 4. 函数体：
  def add(x: int, y: int) -> int:
    return x + y
- 5. 功能解释：
  这个函数定义了一个名为add的函数，它接受两个参数x和y，这两个参数都是整数类型。函数的功能是将这两个整数相加，并返回结果。
- 6. 示例代码：
  add(3,4) → 7
  add(-1,2) → 1


## f-string 和 jinja2 模板提示词格式化

In [4]:
from langchain.prompts import PromptTemplate
# 使用f-string模板提示词格式化
f_string_template = """The answer is {answer}"""
f_string_prompt = PromptTemplate.from_template(f_string_template)
print(f_string_prompt.format(answer=42))

The answer is 42


In [11]:
from langchain.prompts import PromptTemplate
# 适应 jinia2 模板引擎
jinja2_template = "The answer is {{ answer }}"
jinja2_prompt = PromptTemplate.from_template(jinja2_template, template_format="jinja2")
print(jinja2_prompt.format(answer=42))

The answer is 42


______
## 组合模板

- final proprompt  生成最终的提示词
- pipeline prompt template 用于组合多个prompts

In [ ]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

### 三层提示词设计
- 外层prompt: 指定输入和输出格式，以及模板引擎
- 中间层prompt：指定模型的参数
- 最终层prompt: 组合多个prompts
  
 1. 定义外层prompt template   
 2. 定义中间层prompt template 
 3. 组合两个提示词

In [16]:
# final prompt 由一系列变量构成
FINAL_PROMPT = """
{character},
{behavior},
{prohibition}
"""

full_prompt_template = PromptTemplate.from_template(FINAL_PROMPT)
print(full_prompt_template)

input_variables=['behavior', 'character', 'prohibition'] input_types={} partial_variables={} template='\n{character},\n{behavior},\n{prohibition}\n'


- 第一层提示词设计 「character」

In [33]:
chara_prompt = """你是{persion}, 你是公司总裁"""
chara_prompt_template = PromptTemplate.from_template(chara_prompt)
# print(chara_prompt_template)

- 第二层提示词设计 「behavior」

In [32]:
behavior_prompt = """你遵从以下行为：{behavior_list}"""
behavior_prompt_template = PromptTemplate.from_template(behavior_prompt)
# print(behavior_prompt_template)

- 第三层提示词设计 「prohibition」

In [31]:
prohibition_prompt = """禁止使用以下词汇：{prohibition_list}"""
prohibition_prompt_template = PromptTemplate.from_template(prohibition_prompt)
# print(behavior_prompt_template)

- 组合提示词
```

```

In [40]:
"""
A PipelinePrompt consists of two main parts:
- final_prompt: This is the final prompt that is returned
- pipeline_prompts: This is a list of tuples, consisting of
    a string (name) and a Prompt Template. Each PromptTemplate
    will be formatted and then passed to future prompt templates
    as a variable with the same name as name
"""

pipeline_prompt = (
   ("character", chara_prompt_template),
   ("behavior", behavior_prompt_template),
   ("prohibition", prohibition_prompt_template))
# print(pipeline_prompt)

# pipline_prompt = PipelinePromptTemplate(final_prompt=FINAL_PROMPT, pipeline_prompts=pipeline_prompt)

# 组合参数
my_input = {
    "persion": "星际大亨",
    "behavior_list": "不准打人",
    "prohibition_list": ["打人","骂人"]
}


for key, pt in pipeline_prompt:
    my_input[key] = pt.invoke(my_input).to_string()
my_output = full_prompt_template.invoke(my_input)
print(my_output)

text="\n你是星际大亨, 你是公司总裁,\n你遵从以下行为：不准打人,\n禁止使用以下词汇：['打人', '骂人']\n"
